In [126]:
import tensorflow as tf
from tensorflow import tensordot, expand_dims
from tensorflow.keras import layers, Model, initializers, regularizers, activations, constraints, Input

from tensorflow.keras.backend import expand_dims, repeat_elements, sum
import os

In [127]:
class MMoE(tf.keras.Model):
    '''
    @param units MMoE隐藏层单元数
    @param num_experts MMoE专家数
    @param num_tasks 下游任务数
    @param use_expert_bias 是否用用expert的bias
    @param use_gate_bias 是否用用gat的bias
    @param expert_activation expert的激活函数
    @param gate_activation gated的激活函数
    @param expert_bias_initializer expert bias的初始化函数
    @param gate_bias_initializer gate bias的初始化函数
    @param expert_bias_regularizer expert bias的正则化方式
    @param gate_bias_regularizer gate bias的正则化方式
    @param expert_bias_constraint expert bias的约束
    @param gate_bias_constraint gate bias的约束
    @param expert_kernel_initializer expert权重初始化
    @param gate_kernel_initializer gate权重初始化
    @param expert_kernel_regularizer expert权重正则化
    @param gate_kernel_regularizer gate权重正则化
    @param expert_kernel_constraint expert权重约束
    @param gate_kernel_constraint gate权重约束
    @param activity_regularizery activate正则化函数
    @param kwargs Layer类附加参数
    '''
    def __init__(self, units, num_experts, num_tasks, 
                 use_expert_bias=True,use_gate_bias=True,expert_activation='relu', gate_activation='softmax',
                 expert_bias_initializer='zeros',gate_bias_initializer='zeros',expert_bias_regularizer=None, 
                 gate_bias_regularizer=None, expert_bias_constraint=None,gate_bias_constraint=None,
                 expert_kernel_initializer='VarianceScaling', gate_kernel_initializer='VarianceScaling',
                 expert_kernel_regularizer=None,gate_kernel_regularizer=None,expert_kernel_constraint=None,
                 gate_kernel_constraint=None,activity_regularizer=None, **kwargs):
        super(MMoE, self).__init__(**kwargs)
        
        self.units = units
        self.num_experts = num_experts
        self.num_tasks = num_tasks
        
        # Weight parameter
        self.expert_kernels = None
        self.gate_kernels = None
        self.expert_kernel_initializer = initializers.get(expert_kernel_initializer)
        self.gate_kernel_initializer = initializers.get(gate_kernel_initializer)
        self.expert_kernel_regularizer = regularizers.get(expert_kernel_regularizer)
        self.gate_kernel_regularizer = regularizers.get(gate_kernel_regularizer)
        self.expert_kernel_constraint = constraints.get(expert_kernel_constraint)
        self.gate_kernel_constraint = constraints.get(gate_kernel_constraint)

        # Activation parameter
        #self.expert_activation = activations.get(expert_activation)
        self.expert_activation = expert_activation
        self.gate_activation = gate_activation

        # Bias parameter
        self.expert_bias = None
        self.gate_bias = None
        self.use_expert_bias = use_expert_bias
        self.use_gate_bias = use_gate_bias
        self.expert_bias_initializer = initializers.get(expert_bias_initializer)
        self.gate_bias_initializer = initializers.get(gate_bias_initializer)
        self.expert_bias_regularizer = regularizers.get(expert_bias_regularizer)
        self.gate_bias_regularizer = regularizers.get(gate_bias_regularizer)
        self.expert_bias_constraint = constraints.get(expert_bias_constraint)
        self.gate_bias_constraint = constraints.get(gate_bias_constraint)

        # Activity parameter
        self.activity_regularizer = regularizers.get(activity_regularizer)
        
        self.expert_layers = []
        self.gate_layers = []
        
        for i in range(self.num_experts):
            self.expert_layers.append(layers.Dense(self.units, activation=self.expert_activation,
                                                   use_bias=self.use_expert_bias,
                                                   kernel_initializer=self.expert_kernel_initializer,
                                                   bias_initializer=self.expert_bias_initializer,
                                                   kernel_regularizer=self.expert_kernel_regularizer,
                                                   bias_regularizer=self.expert_bias_regularizer,
                                                   activity_regularizer=None,
                                                   kernel_constraint=self.expert_kernel_constraint,
                                                   bias_constraint=self.expert_bias_constraint))
        for i in range(self.num_tasks):
            self.gate_layers.append(layers.Dense(self.num_experts, activation=self.gate_activation,
                                                 use_bias=self.use_gate_bias,
                                                 kernel_initializer=self.gate_kernel_initializer,
                                                 bias_initializer=self.gate_bias_initializer,
                                                 kernel_regularizer=self.gate_kernel_regularizer,
                                                 bias_regularizer=self.gate_bias_regularizer, activity_regularizer=None,
                                                 kernel_constraint=self.gate_kernel_constraint,
                                                 bias_constraint=self.gate_bias_constraint))
    def call(self, inputs):
        expert_outputs, gate_outputs, final_outputs = [], [], []
        for expert_layer in self.expert_layers:
            expert_output = expand_dims(expert_layer(inputs), axis=2)
            #print(expert_output.shape)
            expert_outputs.append(expert_output)
        expert_outputs = tf.concat(expert_outputs,2)
        #print(expert_outputs.shape)
        
        for gate_layer in self.gate_layers:
            gate_outputs.append(gate_layer(inputs))
            
        for gate_output in gate_outputs:
            #print('Gate Ouput')
            #print(gate_output.shape)
            expanded_gate_output = expand_dims(gate_output, axis=1)
            #print(expanded_gate_output.shape)
            weighted_expert_output = expert_outputs * repeat_elements(expanded_gate_output, self.units, axis=1)
            #print(weighted_expert_output.shape)
            final_outputs.append(sum(weighted_expert_output, axis=2))
        return final_outputs

In [128]:
input_layers = layers.Input(shape=(499,))
mmoe_layer = MMoE(units=4, num_experts=8, num_tasks=2)(input_layers)

In [129]:
output_layers = []
for index, task_layer in enumerate(mmoe_layer):
    tower_layer = layers.Dense(units=8, activation='relu', kernel_initializer=tf.keras.initializers.VarianceScaling())(task_layer)
    output_layer = layers.Dense(units=2, name=str(index), activation='softmax', kernel_initializer=tf.keras.initializers.VarianceScaling())(tower_layer)
    output_layers.append(output_layer)


In [130]:
model = tf.keras.Model(inputs=[input_layers], outputs=output_layers)

In [131]:
model.summary()

Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         [(None, 499)]             0         
Total params: 0
Trainable params: 0
Non-trainable params: 0
_________________________________________________________________


In [132]:
input_layer = Input(shape=(499,))#[499,]


    # Set up MMoE layer
mmoe_layers = MMoE(units=4, num_experts=8, num_tasks=2)(input_layer)


output_layers = []
output_dict = {
    '0':'income',
    '1':'marital'
}
    # Build tower layer from MMoE layer
for index, task_layer in enumerate(mmoe_layers):
    tower_layer = layers.Dense(units=8, activation='relu', kernel_initializer=tf.keras.initializers.VarianceScaling())(task_layer)
    output_layer = layers.Dense(units=2, name=output_dict[str(index)], activation='softmax',kernel_initializer=tf.keras.initializers.VarianceScaling())(tower_layer)
    output_layers.append(output_layer)

    # Compile model
model = Model(inputs=[input_layer], outputs=output_layers)

    # Print out model architecture summary
model.summary()

Model: "model_4"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_6 (InputLayer)            [(None, 499)]        0                                            
__________________________________________________________________________________________________
m_mo_e_22 (MMoE)                [(None, 4), (None, 4 24000       input_6[0][0]                    
__________________________________________________________________________________________________
dense_231 (Dense)               (None, 8)            40          m_mo_e_22[0][0]                  
__________________________________________________________________________________________________
dense_232 (Dense)               (None, 8)            40          m_mo_e_22[0][1]                  
____________________________________________________________________________________________

In [133]:
'''
    将csv文件列表处理为TFDataSet
    @param filenames csv文件名列表
    @param n_reader interleave cycle_length
    @param batch_size batch_size
    @param n_parse_threads map的并行数
    @param shuffle_buffer_size shuffer_size
    @return dataset TFDataset
'''
def csv_reader_dataset(filenames, n_readers=5, batch_size=32, n_parse_threads=5, shuffle_buffer_size=10000):
    dataset = tf.data.Dataset.list_files(filenames)
    dataset = dataset.interleave(
        lambda filename : tf.data.TextLineDataset(filename),
        cycle_length=n_readers
    )
    dataset.shuffle(shuffle_buffer_size)
    dataset = dataset.map(parse_csv_line, num_parallel_calls=n_parse_threads)
    dataset = dataset.batch(batch_size)
    return dataset

In [134]:
'''
    将csv数据每一行转换为特征值和标签值
    @param line line
    @param n_feature 特征纬度
    @return x 特征
    @return y 标签
'''
def parse_csv_line(line, n_feature=503):
    defs = [tf.constant(np.nan)] * n_feature
    parsed_field = tf.io.decode_csv(line, record_defaults=defs)
    x = tf.stack(parsed_field[0:-4])
    y_income = tf.stack(parsed_field[-4:-2])
    y_marital = tf.stack(parsed_field[-2:])
    return x, (y_income, y_marital)

In [135]:
'''
    通过前缀获取文件列表
    @param source_dir 源文件目录
    @param prefix_name 前缀名
    @return result list
'''
def get_filename_by_prefix(source_dir,prefix_name):
    all_files = os.listdir(source_dir)
    results = []
    for filename in all_files:
        if filename.startswith(prefix_name):
            results.append(os.path.join(source_dir, filename))
    return results
source_dir='/Users/lizhen/Code/data_set/census-income/generate_csv/'
train_filenames = get_filename_by_prefix(source_dir, 'train')
valid_filenames = get_filename_by_prefix(source_dir, 'val')
test_filenames = get_filename_by_prefix(source_dir, 'test')

In [140]:
metrics = [
            tf.keras.metrics.TruePositives(name='tp'),
            tf.keras.metrics.FalsePositives(name='fp'),
            tf.keras.metrics.TrueNegatives(name='tn'),
            tf.keras.metrics.FalseNegatives(name='fn'),
            tf.keras.metrics.BinaryAccuracy(name='accuracy'),
            tf.keras.metrics.Precision(name='precision'),
            tf.keras.metrics.Recall(name='recall'),
            tf.keras.metrics.AUC(name='auc'),
            tf.keras.metrics.AUC(name='prauc',curve='PR')
        ]
model.compile(
    loss={'income':'binary_crossentropy','marital':'binary_crossentropy'},
    optimizer=tf.keras.optimizers.Adam(),
    metrics=metrics,
    experimental_run_tf_function=False
)

In [137]:
train_dataset = csv_reader_dataset(train_filenames)
val_dataset = csv_reader_dataset(valid_filenames)

In [138]:
import numpy as np

In [141]:
model.fit(train_dataset, validation_data=val_dataset, epochs=1)

6236/6236 [==============================] - 241s 39ms/step - loss: 0.5481 - income_loss: 0.3852 - marital_loss: 0.1629 - income_tp: 187161.0000 - income_fp: 12362.0000 - income_tn: 187161.0000 - income_fn: 12362.0000 - income_accuracy: 0.9380 - income_precision: 0.9380 - income_recall: 0.9380 - income_auc: 0.9384 - income_prauc: 0.9152 - marital_tp: 187294.0000 - marital_fp: 12229.0000 - marital_tn: 187294.0000 - marital_fn: 12229.0000 - marital_accuracy: 0.9387 - marital_precision: 0.9387 - marital_recall: 0.9387 - marital_auc: 0.9851 - marital_prauc: 0.9848 - val_loss: 0.0000e+00 - val_income_loss: 0.0000e+00 - val_marital_loss: 0.0000e+00 - val_income_tp: 0.0000e+00 - val_income_fp: 0.0000e+00 - val_income_tn: 0.0000e+00 - val_income_fn: 0.0000e+00 - val_income_accuracy: 0.0000e+00 - val_income_precision: 0.0000e+00 - val_income_recall: 0.0000e+00 - val_income_auc: 0.0000e+00 - val_income_prauc: 0.0000e+00 - val_marital_tp: 0.0000e+00 - val_marital_fp: 0.0000e+00 - val_marital_tn: 

In [102]:
for x,y in val_dataset.take(1):
    print(x,y)

tf.Tensor(
[[  4.   0.   0. ...   1.   0.   0.]
 [ 39. 500.   0. ...   1.   0.   0.]
 [ 29.   0.   0. ...   1.   0.   0.]
 ...
 [ 28.   0.   0. ...   1.   0.   0.]
 [ 17.   0.   0. ...   1.   0.   0.]
 [ 34.   0.   0. ...   1.   0.   0.]], shape=(32, 499), dtype=float32) (<tf.Tensor: id=46621, shape=(32, 2), dtype=float32, numpy=
array([[1., 0.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.]], dtype=float32)>, <tf.Tensor: id=46622, shape=(32, 2), dtype=float32, numpy=
array([[0., 1.],
       [1., 0.],
       [1., 0

In [142]:
test_dataset = csv_reader_dataset(test_filenames)

In [143]:
model.evaluate(test_dataset)

   1559/Unknown - 42s 27ms/step - loss: 0.5820 - income_loss: 0.3672 - marital_loss: 0.2148 - income_tp: 46799.0000 - income_fp: 3082.0000 - income_tn: 46799.0000 - income_fn: 3082.0000 - income_accuracy: 0.9382 - income_precision: 0.9382 - income_recall: 0.9382 - income_auc: 0.9410 - income_prauc: 0.9255 - marital_tp: 45783.0000 - marital_fp: 4098.0000 - marital_tn: 45783.0000 - marital_fn: 4098.0000 - marital_accuracy: 0.9178 - marital_precision: 0.9178 - marital_recall: 0.9178 - marital_auc: 0.9737 - marital_prauc: 0.9729

[0.5820121931161844,
 0.36719403,
 0.2148183,
 46799.0,
 3082.0,
 46799.0,
 3082.0,
 0.93821293,
 0.93821293,
 0.93821293,
 0.94098943,
 0.9255163,
 45783.0,
 4098.0,
 45783.0,
 4098.0,
 0.9178445,
 0.9178445,
 0.9178445,
 0.97368747,
 0.9729181]

In [105]:
test_filenames

['/Users/lizhen/Code/data_set/census-income/generate_csv/test_data_09.csv',
 '/Users/lizhen/Code/data_set/census-income/generate_csv/test_data_08.csv',
 '/Users/lizhen/Code/data_set/census-income/generate_csv/test_data_06.csv',
 '/Users/lizhen/Code/data_set/census-income/generate_csv/test_data_07.csv',
 '/Users/lizhen/Code/data_set/census-income/generate_csv/test_data_05.csv',
 '/Users/lizhen/Code/data_set/census-income/generate_csv/test_data_04.csv',
 '/Users/lizhen/Code/data_set/census-income/generate_csv/test_data_00.csv',
 '/Users/lizhen/Code/data_set/census-income/generate_csv/test_data_01.csv',
 '/Users/lizhen/Code/data_set/census-income/generate_csv/test_data_03.csv',
 '/Users/lizhen/Code/data_set/census-income/generate_csv/test_data_02.csv']